In [1]:
import pandas as pd
import pytz
import datetime
import os

In [2]:
# Get tables for individual timezones
ipl_schedule = pd.read_csv("data/ipl_schedule.csv", index_col=0)

dt_objs = []
for i in range(1, len(ipl_schedule)+1) : 
    dt_str = ipl_schedule.loc[i, "Date"] + " " + ipl_schedule.loc[i, "Time"]
    if ipl_schedule.loc[i, "Date"] == "TBD":
        dt_objs.append(None)   
    else:
        dt_objs.append(pytz.timezone('Asia/Kolkata').localize(datetime.datetime.strptime(dt_str, '%d %B %Y %I:%M %p')))
        
for tz in pytz.all_timezones:
    ipl_schedule_new = ipl_schedule.copy()
    for i in range(1, len(ipl_schedule_new)+1):
        if ipl_schedule_new.loc[i, "Date"] != "TBD":
            dt_obj_new = dt_objs[i-1].astimezone(pytz.timezone(tz))
            ipl_schedule_new.loc[i, "Date"] = dt_obj_new.strftime("%d %b %Y")
            ipl_schedule_new.loc[i, "Week Day"] = dt_obj_new.strftime("%A")
            ipl_schedule_new.loc[i, "Time"] = dt_obj_new.strftime("%I:%M %p")

    if "/" in tz:
        continent = tz.split("/")[0]
        if not os.path.exists("data/"+continent):
            os.mkdir("data/"+continent)
        ipl_schedule_new.to_csv("data/{0}/ipl_schedule_{1}.csv".format(continent, tz.replace("/","_")))
    else:
        ipl_schedule_new.to_csv("data/ipl_schedule_{0}.csv".format(tz.replace("/","_")))

In [3]:
# Get data for all timezones in one document

ipl_schedule = pd.read_csv("data/ipl_schedule.csv", index_col=0)

for tz in pytz.all_timezones:
    new_tz_data=[]
    for i in range(1, len(ipl_schedule)+1) : 
        dt_str = ipl_schedule.loc[i, "Date"] + " " + ipl_schedule.loc[i, "Time"]
        if ipl_schedule.loc[i, "Date"] == "TBD":
            new_tz_data.append(dt_str)
        else:
            dt_obj = datetime.datetime.strptime(dt_str, '%d %B %Y %I:%M %p')
            dt_obj_in = pytz.timezone('Asia/Kolkata').localize(dt_obj)
            dt_obj_new = dt_obj_in.astimezone(pytz.timezone(tz))
            dt_str_new = dt_obj_new.strftime("%a %d-%b %I:%M %p")
            new_tz_data.append(dt_str_new)

    ipl_schedule[tz] =  new_tz_data
    
ipl_schedule.to_csv("data/ipl_schedule_all_timezones.csv")